In [16]:
#!/usr/bin/env python
#_*_coding:utf-8_*_

import re, sys, os, platform
import math
pPath = os.getcwd()
sys.path.append(pPath)


def minSequenceLengthWithNormalAA(fastas):
	minLen = 10000
	for i in fastas:
		if minLen > len(re.sub('-', '', i[1])):
			minLen = len(re.sub('-', '', i[1]))
	return minLen

def Rvalue(aa1, aa2, AADict, Matrix):
	return sum([(Matrix[i][AADict[aa1]] - Matrix[i][AADict[aa2]]) ** 2 for i in range(len(Matrix))]) / len(Matrix)

def PAAC(fastas, lambdaValue=30, w=0.05, **kw):
	if minSequenceLengthWithNormalAA(fastas) < lambdaValue + 1:
		print('Error: all the sequence length should be larger than the lambdaValue+1: ' + str(lambdaValue + 1) + '\n\n')
		return 0

	dataFile = re.sub('codes$', '', pPath + r'\..\data\PAAC.txt')
	with open(dataFile) as f:
		records = f.readlines()
	AA = ''.join(records[0].rstrip().split()[1:])
	AADict = {}
	for i in range(len(AA)):
		AADict[AA[i]] = i
	AAProperty = []
	AAPropertyNames = []
	for i in range(1, len(records)):
		array = records[i].rstrip().split() if records[i].rstrip() != '' else None
		AAProperty.append([float(j) for j in array[1:]])
		AAPropertyNames.append(array[0])

	AAProperty1 = []
	for i in AAProperty:
		meanI = sum(i) / 20
		fenmu = math.sqrt(sum([(j-meanI)**2 for j in i])/20)
		AAProperty1.append([(j-meanI)/fenmu for j in i])

	encodings = []
	header = ['#']
	for aa in AA:
		header.append('Xc1.' + aa)
	for n in range(1, lambdaValue + 1):
		header.append('Xc2.lambda' + str(n))
	encodings.append(header)

	for i in fastas:
		name, sequence = i[0], re.sub('\*', '', i[1])
		code = [name]
		theta = []
		for n in range(1, lambdaValue + 1):
			theta.append(
				sum([Rvalue(sequence[j], sequence[j + n], AADict, AAProperty1) for j in range(len(sequence) - n)]) / (
				len(sequence) - n))
		myDict = {}
		for aa in AA:
			myDict[aa] = sequence.count(aa)
		code = code + [myDict[aa] / (1 + w * sum(theta)) for aa in AA]
		code = code + [(w * j) / (1 + w * sum(theta)) for j in theta]
		encodings.append(code)
	return encodings

In [13]:
sequence = []
with open('../../data/sequences.txt', 'r') as file_seq:
    for line in file_seq:
        line = line.strip()
        fasta = line.split(" ")
        sequence.append([fasta[0], fasta[2]])
sequence = sequence[:5570]

In [14]:
for (name, seq) in sequence:
	if len(seq)<30:
		print(name, len(seq))

YDL133C-A 26
YDL184C 26
YGR174W-A 29
YOR302W 26


In [21]:
result = PAAC(sequence, order=None, lambdaValue=20)
print(len(result))
print(result[0])
print(result[1])

5571
['#', 'Xc1.A', 'Xc1.R', 'Xc1.N', 'Xc1.D', 'Xc1.C', 'Xc1.Q', 'Xc1.E', 'Xc1.G', 'Xc1.H', 'Xc1.I', 'Xc1.L', 'Xc1.K', 'Xc1.M', 'Xc1.F', 'Xc1.P', 'Xc1.S', 'Xc1.T', 'Xc1.W', 'Xc1.Y', 'Xc1.V', 'Xc2.lambda1', 'Xc2.lambda2', 'Xc2.lambda3', 'Xc2.lambda4', 'Xc2.lambda5', 'Xc2.lambda6', 'Xc2.lambda7', 'Xc2.lambda8', 'Xc2.lambda9', 'Xc2.lambda10', 'Xc2.lambda11', 'Xc2.lambda12', 'Xc2.lambda13', 'Xc2.lambda14', 'Xc2.lambda15', 'Xc2.lambda16', 'Xc2.lambda17', 'Xc2.lambda18', 'Xc2.lambda19', 'Xc2.lambda20']
['YDR143C', 15.86967934282538, 13.885969424972208, 31.73935868565076, 13.885969424972208, 2.380451901423807, 12.299001490689669, 14.282711408542841, 16.66316330996665, 5.157645786418248, 7.93483967141269, 12.695743474260304, 5.554387769988883, 1.1902259507119035, 6.744613720700786, 17.85338926067855, 31.73935868565076, 17.85338926067855, 0.793483967141269, 3.173935868565076, 10.315291572836497, 0.028234574505726494, 0.02951695024168461, 0.028293129499611866, 0.029734482629103116, 0.03014755356

In [23]:
import csv

with open('../../data/Embedings/paac.csv', 'w', newline='\n') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerows(result[1:])